# 한글-Claude-v2 Model: Conversational Interface - Chatbot with Claude LLM

> 이 노트북은 SageMaker Studio의 **`Data Science 3.0`** 커널과 잘 작동합니다.

> **SageMaker Notebook Instance** 를 이용해 실습을 진행하신다면, **JupyterLab이 아닌 Jupyter**에서 실행하시기 바랍니다.

이 노트북에서는 Amazon Bedrock의 기본 모델 (FM) 을 사용하여 챗봇을 구축할 것입니다.사용 사례에서는 Claude를 챗봇 구축을 위한 FM으로 사용합니다.

---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

## 개요

챗봇 및 가상 어시스턴트와 같은 대화형 인터페이스를 사용하여 고객의 사용자 경험을 향상시킬 수 있습니다. 챗봇은 자연어 처리 (NLP) 및 기계 학습 알고리즘을 사용하여 사용자 쿼리를 이해하고 이에 응답합니다.챗봇은 고객 서비스, 판매, 전자 상거래와 같은 다양한 애플리케이션에서 사용되어 사용자에게 빠르고 효율적인 응답을 제공할 수 있습니다.웹사이트, 소셜 미디어 플랫폼 및 메시징 앱과 같은 다양한 채널을 통해 액세스할 수 있습니다.


## Amazon Bedrock을 사용하는 챗봇

![Amazon Bedrock - Conversational Interface](./img/chatbot_bedrock.png)


## 사용 사례
- 1.**챗봇 (기본)** - FM 모델을 사용하는 제로샷 챗봇
- 2.**프롬프트를 사용하는 챗봇** - 템플릿 (Langchain) - 프롬프트 템플릿에 일부 컨텍스트가 제공된 챗봇
- 3.**페르소나가 있는 챗봇** - 정의된 역할을 가진 챗봇. 즉, 커리어 코치와 인간 상호작용
- 4.**컨텍스트 인식 챗봇** - 임베딩을 생성하여 외부 파일을 통해 컨텍스트를 전달합니다.

## Amazon Bedrock으로 챗봇을 구축하기 위한 랭체인 프레임워크
챗봇과 같은 대화형 인터페이스에서는 단기적 수준뿐만 아니라 장기적 수준에서도 이전 상호 작용을 기억하는 것이 매우 중요합니다.

LangChain은 메모리 구성 요소를 두 가지 형태로 제공합니다.먼저 LangChain은 이전 채팅 메시지를 관리하고 조작하기 위한 도우미 유틸리티를 제공합니다.모듈식으로 설계되어 사용 방식에 관계없이 유용하게 사용할 수 있습니다.둘째, LangChain은 이러한 유틸리티를 체인에 통합하는 쉬운 방법을 제공합니다.
이를 통해 다양한 유형의 추상화를 쉽게 정의하고 상호 작용할 수 있으므로 강력한 챗봇을 쉽게 구축할 수 있습니다.

## 컨텍스트를 활용한 챗봇 구축하기 - 핵심 요소

컨텍스트 인식 챗봇을 구축하는 첫 번째 프로세스는 컨텍스트에 대한**임베딩을 생성**하는 것입니다.일반적으로 임베딩 모델을 통해 실행되고 일종의 벡터 저장소에 저장될 임베딩을 생성하는 통합 프로세스가 있습니다.이 예제에서는 이를 위해 Amazon Titan 임베딩 모델을 사용하고 있습니다.

![Embeddings](./img/embeddings_lang.png)

두 번째 프로세스는 사용자 요청 오케스트레이션, 상호 작용, 호출 및 결과 반환입니다.

![Chatbot](./img/chatbot_lang.png)

## 아키텍처 [컨텍스트 인식 챗봇]
![4](./img/context-aware-chatbot.png)


### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.


이 노트북에는 몇 가지 추가 종속성도 필요합니다.

- [FAISS](https://github.com/facebookresearch/faiss), 벡터 임베딩 저장용
- [iPyWidgets](https://ipywidgets.readthedocs.io/en/stable/), 노트북의 대화형 UI 위젯용
- [PyPDF](https://pypi.org/project/pypdf/), PDF 파일 처리용

In [ ]:
import boto3
region = boto3.Session().region_name
opensearch = boto3.client('opensearch', region)

%store -r opensearch_user_id opensearch_user_password domain_name opensearch_domain_endpoint

try:
    opensearch_user_id
    opensearch_user_password
    domain_name
    opensearch_domain_endpoint
   
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

# 1. Bedrock Client 생성

In [ ]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

# 2. 챗봇(기본 - 컨텍스트 없음)

우리는 LangChain의 [CoversationChain](https://python.langchain.com/en/latest/modules/models/llms/integrations/bedrock.html?highlight=ConversationChain#using-in-a-conversation-chain)을 사용하여 시작합니다. 대화. 또한 메시지 저장을 위해 [ConversationBufferMemory](https://python.langchain.com/en/latest/modules/memory/types/buffer.html)를 사용합니다. 메시지 목록으로 기록을 얻을 수도 있습니다(채팅 모델에서 매우 유용합니다).

챗봇은 이전 상호작용을 기억해야 합니다. 대화 기억을 통해 우리는 그렇게 할 수 있습니다. 대화형 메모리를 구현하는 방법에는 여러 가지가 있습니다. LangChain의 맥락에서 이들은 모두 ConversationChain 위에 구축됩니다.

**참고:** 모델 출력은 비결정적입니다.

In [ ]:
from utils.chat import chat_utils
from langchain.llms.bedrock import Bedrock
from langchain.chains import ConversationChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512,
        "temperature": 0,
        "top_k": 250,
        "top_p": 0.999,
        "stop_sequences": ["\n\nHuman:", "\n\nUser:"]
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

## Conversation momory
* **[ConversationBufferMemory](https://python.langchain.com/docs/modules/memory/types/buffer)**
    - This memory allows for storing messages and then extracts the messages in a variable.
* **[ConversationBufferWindowMemory](https://python.langchain.com/docs/modules/memory/types/buffer_window)**
    - It keeps a list of the interactions of the conversation over time.
    - It only uses the last K interactions.
    - This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.
* **[ConversationSummaryBufferMemory](https://python.langchain.com/docs/modules/memory/types/summary_buffer)**
    - It maintains a summary of previous messages.
    - It combines the two ideas.
    - It keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions it compiles them into a summary and uses both. 
    - It uses token length rather than number of interactions to determine when to flush interactions.

In [ ]:
memory = chat_utils.get_memory(
    memory_type="ConversationBufferMemory",
    memory_key="history"
)

conversation = ConversationChain(
    llm=llm_text,
    verbose=True,
    memory=memory
)

In [ ]:
if llm_text.streaming:
    response = conversation.predict(input="안녕하세요?")
else:
    print_ww(conversation.predict(input="안녕하세요?"))

### 결과 분석

여기서 무슨 일이 일어나는가? 우리는 "안녕하세요!"라고 말했습니다. 모델은 몇 가지 대화를 나눴습니다. 이는 Langchain ConversationChain에서 사용하는 기본 프롬프트가 Claude에 맞게 잘 설계되지 않았기 때문입니다. 
- [효과적인 클로드 프롬프트](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)는 `\n\nHuman\n\nAassistant:`로 끝나야 합니다. 이 문제를 해결해 보겠습니다.

Claude의 프롬프트 작성 방법에 대해 자세히 알아보려면 [Anthropic 문서](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)를 확인하세요.

### Reset memory

In [ ]:
chat_utils.clear_memory(
    chain=conversation
)
print_ww(memory.load_memory_variables({}))

# 3. 프롬프트 템플릿(Langchain)을 이용한 챗봇

LangChain은 프롬프트를 쉽게 구성하고 작업할 수 있도록 여러 클래스와 기능을 제공합니다. [PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html) 클래스를 사용하여 f-string 템플릿에서 프롬프트를 구성하겠습니다.

**[TIP] Prompt의 instruction의 경우 한글보다 **영어**로 했을 때 더 좋은 결과를 얻을 수 있습니다.**

In [ ]:
from langchain import PromptTemplate

In [ ]:
# turn verbose to true to see the full logs and documents
conversation = ConversationChain(
    llm=llm_text,
    verbose=False,
    memory=memory
)

claude_prompt = PromptTemplate(
        input_variables=["history", 'input'],
        template="""
        \n\nHuman: Here's a friendly conversation between a user and an AI.
        The AI is talkative and provides lots of contextualized details.
        If it doesn't know, it will honestly say that it doesn't know the answer to the question.

        Current conversation:
        {history}

        User:
        {input}

        \n\nAssistant:
        """
)

print("claude_prompt: \n", claude_prompt)

In [ ]:
conversation.prompt = claude_prompt

In [ ]:
chat_utils.get_tokens(
    chain=conversation,
    prompt="안녕하세요?"
)

In [ ]:
if llm_text.streaming:
    response = conversation.predict(input="안녕하세요?")
else:
    print_ww(conversation.predict(input="안녕하세요?"))

#### (1) 새로운 질문

모델이 초기 메시지로 응답했습니다. 몇 가지 질문을 해보겠습니다.

In [ ]:
if llm_text.streaming:
    response = conversation.predict(input="새로운 정원을 시작하는 방법에 대한 몇 가지 팁을 알려주세요.")
else:
    print_ww(conversation.predict(input="새로운 정원을 시작하는 방법에 대한 몇 가지 팁을 알려주세요."))

#### Check momory

In [ ]:
print_ww(memory.load_memory_variables({}))

#### (2) 질문을 토대로 작성

모델이 이전 대화를 이해할 수 있는지 확인하기 위해 정원이라는 단어를 언급하지 않고 질문해 보겠습니다.

In [ ]:
if llm_text.streaming:
    response = conversation.predict(input="좋아요. 토마토에도 어울릴까요?")
else:
    print_ww(conversation.predict(input="좋아요. 토마토에도 어울릴까요?"))

In [ ]:
print_ww (memory.load_memory_variables({}))

#### (3) 대화를 마치며

In [ ]:
if llm_text.streaming:
    response = conversation.predict(input="그게 다야, 고마워!")
else:
    print_ww(conversation.predict(input="그게 다야, 고마워!"))

# 4. ipywidgets를 사용한 대화형 세션

다음 유틸리티 클래스를 사용하면 Claude와 보다 자연스러운 방식으로 상호 작용할 수 있습니다. 입력창에 질문을 적고 클로드의 답변을 받습니다. 그러면 대화를 계속할 수 있습니다.

In [ ]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()

        if "ConversationChain" in str(type(self.qa)):
            self.streaming = self.qa.llm.streaming
        elif "ConversationalRetrievalChain" in str(type(self.qa)):
            self.streaming = self.qa.combine_docs_chain.llm_chain.llm.streaming

    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                if self.streaming:
                    response = f"AI:{result}"
                else:
                    print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

#### (1) 채팅을 시작해 보겠습니다. 다음 질문을 테스트할 수도 있습니다.

1. 농담 하나 해줘
2. 또 다른 농담을 들려주세요
3. 첫 번째 농담은 무엇이었나요?
4. 첫 번째 농담과 같은 주제로 또 다른 농담을 할 수 있나요?

위의 4가지를 순서대로 아래에 입력하시고, "Send" 버튼을 눌러 보세요.

In [ ]:
chat = ChatUX(conversation)

In [ ]:
chat = ChatUX(conversation)
chat.start_chat()

# 5.페르소나를 활용한 챗봇

AI 비서가 커리어 코치 역할을 하게 됩니다. 
- 역할극 대화에서는 채팅을 시작하기 전에 사용자 메시지를 설정해야 합니다. ConversationBufferMemory는 대화 상자를 미리 채우는 데 사용됩니다.

### (1) ConversationChain 생성 필요한 메모리 초기화, Bedrock Claude 설정

In [ ]:
# llm
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 1000,
        "temperature": 0,
        "top_k": 250,
        "top_p": 0.999,
        "stop_sequences": ["\n\nHuman:"]
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# memory
# store previous interactions using ConversationalBufferMemory and add custom prompts to the chat.
memory = chat_utils.get_memory(
    memory_type="ConversationBufferMemory",
    memory_key="history"
)

memory.chat_memory.add_user_message("당신은 직업 코치로 활동하게 될 것입니다. 귀하의 목표는 사용자에게 직업 조언을 제공하는 것입니다")
memory.chat_memory.add_ai_message("나는 직업 코치이며 직업에 대한 조언을 제공합니다")

# conversation chain
conversation = ConversationChain(
    llm=llm_text,
    verbose=True,
    memory=memory
)
print(conversation)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude
claude_prompt = PromptTemplate.from_template("""
\n\nHuman: Here's a friendly conversation between a user and an AI.
The AI is talkative and provides lots of contextualized details.
If it doesn't know, it will honestly say that it doesn't know the answer to the question.

Current conversation:
{history}

User: {input}

\n\nAssistant:
"""
)


print("claude_prompt: \n", claude_prompt)
conversation.prompt = claude_prompt

### (2) 인공지능 관련 직업 질문

In [ ]:
if llm_text.streaming:
    response = conversation.predict(input="“인공지능에 관련된 직업은 어떤 것이 있습니까?")
else:
    print_ww(conversation.predict(input="“인공지능에 관련된 직업은 어떤 것이 있습니까?"))

### (3) 인공지능 관련 직업에 대한 세부 질문

In [ ]:
if llm_text.streaming:
    response = conversation.predict(input="이 직업들은 실제로 무엇을 하는가요? 재미있나요?")
else:
    print_ww(conversation.predict(input="이 직업들은 실제로 무엇을 하는가요? 재미있나요?"))

#### Check memory

In [ ]:
pprint(memory.load_memory_variables({}))

# 6. 맥락을 가진 챗봇

## 상황에 맞는 챗봇
이 사용 사례에서는 Chatbot에게 이전에 본 적이 없는 외부 코퍼스의 질문에 답변하도록 요청합니다. 이를 위해 RAG(Retrieval Augmented Generation)라는 패턴을 적용합니다. 아이디어는 말뭉치를 덩어리로 인덱싱한 다음 덩어리와 질문 사이의 의미론적 유사성을 사용하여 말뭉치의 어느 섹션이 답변을 제공하는 데 관련될 수 있는지 찾는 것입니다. 마지막으로 가장 관련성이 높은 청크가 집계되어 기록을 제공하는 것과 유사하게 ConversationChain에 컨텍스트로 전달됩니다.

**Titan Embeddings Model**을 사용하여 벡터를 생성하겠습니다. 그런 다음 이 벡터는 메모리 내 벡터 데이터 저장소를 제공하는 Amazon OpenSearch에 저장됩니다. 챗봇이 질문을 받으면 OpenSearch에 질문을 쿼리하고 의미상 가장 가까운 텍스트를 검색합니다. 이것이 우리의 대답이 될 것입니다.

## 6.1 2022년 아마존 주주 서한 문서로 구현 (PDF 문서)

### Amazon Titan Embedding 모델 사용 
- model_id="amazon.titan-embed-text-v1"
- 이 모델은 최대 8,192 Token 입력이 가능합니다.

In [ ]:
from langchain.embeddings import BedrockEmbeddings

In [ ]:
bedrock_embeddings = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id=bedrock_info.get_model_id(
        model_name="Titan-Embeddings-G1"
    )
)

### PyPDFDirectoryLoader 를 통한 PDF 파일 로딩
- chunk_size 는 임베딩 모델의 최대 입력 토큰 (8,192) 수를 고려하여 정해야 합니다.

In [ ]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, SpacyTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.vectorstores import FAISS
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

In [ ]:
loader = PyPDFDirectoryLoader("./data/rag_data_kr_pdf/")

* RecursiveCharacterTextSplitter: charater based splitter

In [ ]:
# documents = loader.load()
# # - in our testing Character split works better with this PDF data set
# text_splitter = RecursiveCharacterTextSplitter(
#     # Set a really small chunk size, just to show.
#     chunk_size=1024,
#     chunk_overlap=256,
#     # separators = ['\n'],
#     separators = ['\n','\n\n']
# )
# docs = text_splitter.split_documents(documents)

In [ ]:
#docs[:3]

* [llmsherpa: layout info. based splitter](https://github.com/nlmatics/llmsherpa)

    - https://blog.llamaindex.ai/mastering-pdfs-extracting-sections-headings-paragraphs-and-tables-with-cutting-edge-parser-faea18870125

Most PDF to text parsers do not provide layout information. Often times, even the sentences are split with arbritrary CR/LFs making it very difficult to find paragraph boundaries. This poses various challenges in chunking and adding long running contextual information such as section header to the passages while indexing/vectorizing PDFs for LLM applications such as retrieval augmented generation (RAG).

LayoutPDFReader solves this problem by parsing PDFs along with hierarchical layout information such as:

Sections and subsections along with their levels.
Paragraphs - combines lines.
Links between sections and paragraphs.
Tables along with the section the tables are found in.
Lists and nested lists.
With LayoutPDFReader, developers can find optimal chunks of text to vectorize, and a solution for limited context window sizes of LLMs.

In [ ]:
import time
from langchain.schema import Document
from llmsherpa.readers import LayoutPDFReader

In [ ]:
llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf("./data/rag_data_kr_pdf/2022-Amazon-Stakeholder-Letter-KR.pdf")

In [ ]:
#doc.json
docs = []
for chunk_info in doc.chunks():
    sentences = " ".join(chunk_info.sentences)
    chunk = Document(
        page_content=sentences,
        metadata={
            "type": "value you want",
            "timestamp": time.time()
        }
    )
    docs.append(chunk)

In [ ]:
docs[:10]

* SpacyTextSplitter: sentence based splitter

In [ ]:
# documents = loader.load()
# text_splitter = SpacyTextSplitter(
#     chunk_size=1024,
#     chunk_overlap=512,
#     separator="", # seperator to be inserted between sentences 
#     pipeline="ko_core_news_md"
# )
# docs = text_splitter.split_documents(documents)

### OpenSearch Client 생성
### 선수 조건
- 아래의 링크를 참조해서 OpenSearch Service 를 생성하고, opensearch_domain_endpoint, http_auth 를 복사해서, 아래 셀의 내용을 대체 하세요.
    - [OpenSearch 생성 가이드](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/2-Lab02-QA-with-RAG/4.rag-fsi-data-workshop/TASK-4_OpenSearch_Creation_and_Vector_Insertion.ipynb)
    
### 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 20_applications/02_qa_chatbot/01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [ ]:
# from utils.proc_docs import get_parameter
# aws_region = 'us-east-1'
# ssm = boto3.client("ssm", aws_region)

# opensearch_domain_endpoint = get_parameter(
#     boto3_clinet = ssm,
#     parameter_name = 'knox_opensearch_domain_endpoint',
# )

# opensearch_user_id = get_parameter(
#     boto3_clinet = ssm,
#     parameter_name = 'knox_opensearch_userid',
# )

# opensearch_user_password = get_parameter(
#     boto3_clinet = ssm,
#     parameter_name = 'knox_opensearch_password',
# )


In [ ]:
region

In [ ]:
from utils.opensearch import opensearch_utils
aws_region = region
http_auth = (opensearch_user_id, opensearch_user_password) # Master username, Master password
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

### OpenSearch 벡터 Indexer 생성
- 랭체인 오프서처 참고 자료
    - [Langchain Opensearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch)

#### 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [ ]:
index_name = "genai-demo-chatbot-index-v1"
index_exists = opensearch_utils.check_if_index_exists(os_client, index_name)

if index_exists:
    opensearch_utils.delete_index(os_client, index_name)
else:
    print("Index does not exist")

#### 인덱스 생성

In [ ]:
from langchain.vectorstores import OpenSearchVectorSearch

In [ ]:
%%time
# by default langchain would create a k-NN index and the embeddings would be ingested as a k-NN vector type
vector_db = OpenSearchVectorSearch.from_documents(
    index_name=index_name,
    documents=docs,
    embedding=bedrock_embeddings,
    opensearch_url=opensearch_domain_endpoint,
    http_auth=http_auth,
    bulk_size=10000,
    timeout=60,
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)

#### 인덱스 확인

In [ ]:
index_info = os_client.indices.get(index=index_name)
pprint(index_info)

### 형태소 분석기 사용하기
- 영어권의 문자들과 다르게 한글, 일본어, 중국어 등은 단순한 공백만으로는 좋은 검색 결과를 얻기 힘듭니다.
- 출시하고라는 단어가 들어간 문서를 출시하고라는 정확히 같은 단어만으로 검색할 수 있다면 답답하겠죠?
- 출시하고라는 단어를 출시, 출시하고 등 다양하게 검색하기 위해서는 형태소 분석기가 필요합니다.
- OpenSearch 에서는 2개의 한국어 analyer를 제공하고 있습니다.
    - 은전한잎 (seunjeon_tokenizer)
        - https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/stemming#
    - Nori (nori_tokenizer)
        - 설명: https://esbook.kimjmin.net/06-text-analysis/6.7-stemming/6.7.2-nori
    - Sample 코드에서는 "Nori"를 기반으로 진행합니다.

### [중요] Nori 사용을 위해서는 nori plugin이 설치되어 있어야 합니다.
* 자세한 사항은 10_Adv_QuestionAnswering - 02_1_KR_RAG_OpenSearch_Keyword.ipynb를 참고해 주세요

#### 인덱싱 수정하기 (형태소 분석시 사용 enablement)

In [ ]:
new_index_name = f'{index_name}-with-tokenizer'
new_index_name

In [ ]:
tokenizer = "nori" #["nori", "seunjeon"]
analyzer_config = {
    "tokenizer": tokenizer,
    "tokenizer_type": f'{tokenizer}_tokenizer',
    "char_filter": ["html_strip"],
    "filter": ["nori_number", "nori_readingform", "lowercase"],
    "decompound_mode": "mixed",
    "discard_punctuation": "true",
    #"user_dictionary_rules": ["c++", "워라밸", "먹방"],
    #"user_dictionary": "analyzers/F255700190"
}

In [ ]:
# Setting for "Nori" Tokenizer (변경 없음)
index_info[index_name]["settings"]["analysis"] = {
    "tokenizer": {
        analyzer_config["tokenizer"]: {
            "type": analyzer_config["tokenizer_type"],
            "decompound_mode": analyzer_config["decompound_mode"],
            "discard_punctuation": analyzer_config["discard_punctuation"],
            #"user_dictionary_rules": analyzer_config["user_dictionary_rules"],
            #"user_dictionary": analyzer_config["user_dictionary"],
        }
    },
    "analyzer": {
        "my_analyzer": {
            "type": "custom",
            "tokenizer": analyzer_config["tokenizer"],
            "char_filter": analyzer_config["char_filter"],
            "filter": analyzer_config["filter"],
        }
    }
}


# Setting for Columns to be adapted by Tokenizer (tokenizer가 적용될 컬럼에 맞춰서 수정)
index_info[index_name]["mappings"]["properties"]["text"]["analyzer"] = "my_analyzer"
index_info[index_name]["mappings"]["properties"]["text"]["search_analyzer"] = "my_analyzer"

# Setting for vector index column (변경 없음)
index_info[index_name]["settings"]["index"] = {
    "number_of_shards": "5",
    "knn.algo_param": {"ef_search": "512"},
    "knn": "true",
    "number_of_replicas": "2"
}
del index_info[index_name]["aliases"]
new_index_info = index_info[index_name]

In [ ]:
pprint(new_index_info)

#### 형태소 분석기용 인덱서 생성

In [ ]:
index_exists = opensearch_utils.check_if_index_exists(os_client, new_index_name)
if index_exists:
    opensearch_utils.delete_index(os_client, new_index_name)
else:
    print("Index does not exist")

In [ ]:
opensearch_utils.create_index(
    os_client,
    index_name=new_index_name,
    index_body=new_index_info
)

#### Re-indexing

In [ ]:
_reindex = {
    "source": {"index": index_name},
    "dest": {"index": new_index_name}
}
print("_reindex: \n", _reindex)

In [ ]:
os_client.reindex(_reindex)

#### 키워드 검색 체크

In [ ]:
query = "아마존"
query = opensearch_utils.get_query(
    query=query
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, new_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

#### 형태소 분석 결과 확인
"아마존" 확인 <BR>
#### [중요]:  doc_id: 위의 문서 인덱스 정보 확인 후 수정

In [ ]:
doc_id = "1bcc2bc6-de73-486c-a13f-f2a7cf0d10f9"

In [ ]:
os_client.termvectors(index=new_index_name, id=doc_id, fields='text')

### 문서 및 임베딩 확인

In [ ]:
loader = PyPDFDirectoryLoader("./data/rag_data_kr_pdf/")
documents = loader.load()

In [ ]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

In [ ]:
print("docs[0].page_content: \n", docs[0].page_content)

In [ ]:
sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

### Hybrid search

LangChain에서 제공하는 Wrapper 클래스를 사용하여 벡터 데이터베이스 저장소를 쿼리하고 관련 문서를 반환할 수 있습니다. 뒤에서는 RetrievalQA 체인만 실행됩니다.

In [ ]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings

        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )

        print("Bedrock Embeddings Model Loaded")
        
    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)     

In [ ]:
from utils.rag import OpenSearchHybridSearchRetriever
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    os_client=os_client,
    vector_db=vector_db,
    index_name=index_name,
    llm_emb = llm_emb,
    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for rank fusion
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.5, .5], # [for lexical, for semantic], Lexical, Semantic search 결과에 대한 최종 반영 비율 정의

    # option for async search
    async_mode=True,
    
    # option for output
    k=5, # 최종 Document 수 정의
    verbose=False,
)

In [ ]:
query = "아마존은 Generative AI 의 전략이 무엇인가요?"
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print(f'question: {query}')
print(f'response: {search_hybrid_result}', len(search_hybrid_result))

하이브리드(lexical + semantic) 검색이 어떻게 작동하는지 살펴보겠습니다.
1. 먼저 쿼리에 대한 임베딩 벡터를 계산하고
2. 그런 다음 이 벡터를 사용하여 벡터 스토어에서 유사성 검색을 수행한다.
3. document 풀을 기반으로 쿼리에 대한 full text 검색을 수행한다.
4. 두가지 결과를 바탕으로 re-ranking을 한다.
5. 결과를 반환한다.

In [ ]:
v = bedrock_embeddings.embed_query(query)
print(v[0:10], len(v))
for r in search_hybrid_result:
    print_ww(r.page_content)
    print('----')

### 메모리
모든 챗봇에는 사용 사례에 따라 맞춤화된 다양한 옵션을 갖춘 QA 체인이 필요합니다. 그러나 챗봇에서는 모델이 답변을 제공하기 위해 이를 고려할 수 있도록 항상 대화 기록을 보관해야 합니다. 이 예에서는 대화 기록을 유지하기 위해 ConversationBufferMemory와 함께 LangChain의 [ConversationalRetrievalChain](https://python.langchain.com/docs/modules/chains/popular/chat_Vector_db)을 사용합니다.

출처: https://python.langchain.com/docs/modules/chains/popular/chat_Vector_db

뒤에서 무슨 일이 일어나고 있는지 모두 보려면 'verbose'를 'True'로 설정하세요.

In [ ]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
print (CONDENSE_QUESTION_PROMPT.template)

### ConversationRetrievalChain에 사용되는 매개변수
* **retriever**: 우리는 `OpenSearch`를 기반으로 customization한 `OpenSearchHybridSearchRetriever`를 사용했습니다. 이것은 `OpenSearch`를 이용하여 주어진 `query`에 대한 `lexical`, `sematic` search를 기반으로 최종 후보 context 를 선택해 줍니다.

* **메모리**: 이력을 저장하는 메모리 체인

* **condense_question_prompt**: 사용자의 질문이 주어지면 이전 대화와 해당 질문을 사용하여 독립형 질문을 구성합니다.

* **chain_type**: 채팅 기록이 길고 상황에 맞지 않는 경우 이 매개변수를 사용하고 옵션은 `stuff`, `refine`, `map_reduce`, `map-rerank`입니다.

질문이 컨텍스트 범위를 벗어나면 모델은 답을 모른다고 대답합니다.

**참고**: 체인이 어떻게 작동하는지 궁금하다면 `verbose=True` 줄의 주석 처리를 해제하세요.

In [ ]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [ ]:
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 1000,
        "temperature": 0,
        "top_k": 250,
        "top_p": 0.999,
        "stop_sequences": ["\n\nHuman:"]
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [ ]:
# turn verbose to true to see the full logs and documents
from langchain.schema import BaseMessage
from langchain.chains import ConversationalRetrievalChain

In [ ]:
# We are also providing a different chat history retriever which outputs the history as a Claude chat (ie including the \n\n)
_ROLE_MAP = {"human": "\n\nUser: ", "ai": "\n\nAI: "}
def _get_chat_history(chat_history):
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "\n\nUser: " + dialogue_turn[0]
            ai = "\n\nAI: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer

# 이전 대화 내용을 기반으로, 신규 질문을 재구성 하는 prompt
condense_prompt_claude = PromptTemplate.from_template("""
\n\nHuman:
Given the following conversation and a follow up question,
rephrase the follow up question to be a standalone question, in its original language.

Chat History: {chat_history}
Follow Up Input: {question}

\n\nAssistant: Question:"""
)

# memory
# store previous interactions using ConversationalBufferMemory and add custom prompts to the chat.
memory_chain = chat_utils.get_memory(
    memory_type="ConversationBufferMemory",
    memory_key="chat_history",
    return_messages=True
)

opensearch_hybrid_retriever.update_search_params(
    k=5,
    minimum_should_match=0,
    filter=[],
    verbose=False
)

qa = ConversationalRetrievalChain.from_llm(
    llm=llm_text,
    retriever=opensearch_hybrid_retriever,
    memory=memory_chain,
    get_chat_history=_get_chat_history,
    verbose=True,
    condense_question_prompt=condense_prompt_claude,
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# the LLMChain prompt to get the answer. the ConversationalRetrievalChange does not expose this parameter in the constructor
qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
\n\nHuman:

{context}

<q></q> XML 태그 내의 질문에 답하려면 최대 3개의 문장을 사용하세요.

<q>{question}</q>

답변에 XML 태그를 사용하지 마세요.
답변이 context에 없으면 "죄송합니다. 문맥에서 답을 찾을 수 없어서 모르겠습니다."라고 말합니다.

\n\nAssistant:""")

채팅을 해보죠. 아래와 같은 질문을 해보세요. 
1. 무슨 내용의 편지인가요?
2. 미래 전망은 어떤가요?
3. generative ai전략에 대해서 말해 주세요

In [ ]:
from utils.chat import ChatUX

In [ ]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

## Delete OpenSearch Index

In [ ]:
print(index_name, new_index_name)

In [ ]:
for name in [index_name, new_index_name]:

    index_exists = opensearch_utils.check_if_index_exists(os_client, name)
    if index_exists:
        opensearch_utils.delete_index(os_client, name)
    else:
        print("Index does not exist")